# Energy determination of parameters

## Order expansion of parameters

The wave function parameters are expanded in orders of the external field

$$
\kappa_{ai}(F) =
\kappa_{ai}^{(1)} +
\kappa_{ai}^{(2)} + \ldots
$$

such that $\kappa_{ai}= 0$ in the limit of $F = 0$.

Conversely, 

$$
\kappa_{ai}^{(1)} =
\left.
\frac{
d \kappa_{ai}
}{
d F_{\alpha}
}
\right|_{F=0} F_{\alpha} 
$$

## Linear response equation

The first-order response can be determined from the field-derivative of the variational condition

\begin{align*}
0 & =
\Big[
\frac{
d
}{
d F_{\alpha}
} 
\frac{\partial E(\kappa, F)}{\partial \kappa_{bj}^*}
\Big]_{F=0} 
\\ & = 
\Big[
\frac{\partial^2 E(\kappa, F)}{\partial \kappa_{bj}^* \partial F_\alpha}
+
\sum_{ai} 
\Big(
\frac{
\partial^2 E(\kappa, F)
}{
\partial \kappa_{bj}^* \partial \kappa_{ai}
}
\frac{\partial \kappa_{ai}}{\partial F_\alpha} 
+
\frac{
\partial^2 E(\kappa, F)
}{
\partial \kappa_{bj}^* \partial \kappa_{ai}^*
}
\frac{\partial \kappa_{ai}^*}{\partial F_\alpha} 
\Big)
\Big]_{F=0} 
\end{align*}

We here view the $ai$-pair of unoccupied/occupied orbital indices as a compound index $n$ (and similarly for $bj$ and $m$), inviting us to introduce a compact matrix notation to arrive at the linear response equation.

$$
\mathbf{E}^{[2]} \mathbf{N} = - \mathbf{B}^{[1]}
$$

In order, we have introduced:

1. The Hermitian electronic Hessian
$$
\mathbf{E}^{[2]} = 
\begin{pmatrix}
\mathbf{A} & \mathbf{B} \\
\mathbf{B}^* & \mathbf{A}^* \\
\end{pmatrix}
$$
with blocks
$$
A_{mn} =
\left.
\frac{
\partial^2 E(\kappa, F)
}{
\partial \kappa_{m}^* \partial \kappa_{n}
}
\right|_{F = 0} ; \qquad
B_{mn} =
\left.
\frac{
\partial^2 E(\kappa, F)
}{
\partial \kappa_{m}^* \partial \kappa_{n}^*
}
\right|_{F = 0} 
$$

2. The (unknown) response vector
$$
\mathbf{N} =
\begin{pmatrix}
\mathbf{Z} \\
\mathbf{Z}^*
\end{pmatrix}
$$
with 
$$
Z_n = 
\left.
\frac{\partial \kappa_n}{\partial F_\alpha}
\right|_{F=0}
$$

3. And the property gradient
$$
\mathbf{B}^{[1]} =
\begin{pmatrix}
\mathbf{g} \\
\mathbf{g}^* \\
\end{pmatrix}
$$
with
$$
g_m = 
\left.
\frac{\partial^2 E(\kappa, F)}{\partial \kappa_{m}^* \partial F_\alpha}
\right|_{F=0}
$$

## Baker–Campbell–Hausdorff expansion

We have

\begin{align*}
E(\kappa,F) & =
\langle 0 | e^{i\hat{\kappa}} \hat{H} e^{-i\hat{\kappa}} | 0 \rangle
\\ & =
\langle 0 | \hat{H} | 0 \rangle +
i \langle 0 | [\hat{\kappa}, \hat{H}] | 0 \rangle -
\frac{1}{2}
\langle 0 | [\hat{\kappa}, [\hat{\kappa}, \hat{H}]] | 0 \rangle +
\cdots
\end{align*}

and let us adopt the electric-dipole approximation

$$
\hat{H} = \hat{H}_0 - \hat{\mu}_\alpha F_\alpha
$$

It is then straightforward to evaluate that electronic Hessian and the property gradient. We get

\begin{align*}
A_{mn} & = -
\langle 0 | [\hat{a}^\dagger_j \hat{a}_b, 
[\hat{a}^\dagger_a \hat{a}_i, \hat{H}_0]] | 0 \rangle
\\ & =
\langle 0_j^b | \hat{H}_0 | 0_i^a \rangle -
\langle 0 | \hat{H}_0 | 0 \rangle \delta_{ab} \delta_{ij}
\\
B_{mn} & = -
\langle 0 | [\hat{a}^\dagger_j \hat{a}_b, 
[\hat{a}^\dagger_i \hat{a}_a, \hat{H}_0]] | 0 \rangle
\\ & =
- \langle 0_{ij}^{ab} | \hat{H}_0 | 0 \rangle 
( 1 - \delta_{ab} ) ( 1 - \delta_{ij} )
\end{align*}

and

\begin{align*}
g_m & = i
\langle 0 | [\hat{a}^\dagger_j \hat{a}_b, -\hat{\mu}_\alpha ] | 0 \rangle 
\\ & =
- i \langle 0_j^b | \hat{\mu}_\alpha | 0 \rangle
\end{align*}

In [2]:
import numpy as np
import veloxchem as vlx

* Warning * Environment variable OMP_NUM_THREADS not set.
* Warning * Setting OMP_NUM_THREADS to 10.


In [3]:
ethylene_xyz = """
6

C        0.67759997    0.00000000   0.00000000
C       -0.67759997    0.00000000   0.00000000
H        1.21655197    0.92414474   0.00000000
H        1.21655197   -0.92414474   0.00000000
H       -1.21655197   -0.92414474   0.00000000
H       -1.21655197    0.92414474   0.00000000
"""

In [4]:
molecule = vlx.Molecule.read_xyz_string(ethylene_xyz)
basis = vlx.MolecularBasis.read(molecule, "def2-svp", ostream=None)

nocc = molecule.number_of_alpha_electrons()
norb = basis.get_dimension_of_basis(molecule)
nvirt = norb - nocc

print("Number of occupied orbitals :", nocc)
print("Number of virtual orbitals  :", nvirt)
print("Number of molecular orbitals:", norb)

Number of occupied orbitals : 8
Number of virtual orbitals  : 40
Number of molecular orbitals: 48


First, we determine the Hartree–Fock reference state.

In [5]:
scf_drv = vlx.ScfRestrictedDriver()
scf_drv.ostream.mute()

scf_results = scf_drv.compute(molecule, basis)

Second, we get the MO representation of the operator $\hat{\Omega}$.

In [6]:
dipole_drv = vlx.ElectricDipoleIntegralsDriver()
dipole_mats = dipole_drv.compute(molecule, basis)

mu_x_ao = -1.0 * dipole_mats.x_to_numpy()

C = scf_results["C_alpha"]

mu_x = np.einsum("ap, ab, bq -> pq", C, mu_x_ao, C)

Third, we determine the linear response parameters.

In [7]:
lrf_drv = vlx.LinearResponseSolver()
lrf_drv.ostream.mute()

lrf_drv.a_operator = "electric dipole"
lrf_drv.b_operator = "electric dipole"

lrf_drv.a_components = ["x"]
lrf_drv.b_components = ["x"]

lrf_drv.frequencies = [0.0]

lrf_results = lrf_drv.compute(molecule, basis, scf_results)

In [9]:
Z = -0.5 * (
    lrf_results["solutions"][("x", 0.0)].get_full_vector(0)
    + lrf_results["solutions"][("x", 0.0)].get_full_vector(1)
) * np.sqrt(2)

Y = -0.5 * (
    lrf_results["solutions"][("x", 0.0)].get_full_vector(0)
    - lrf_results["solutions"][("x", 0.0)].get_full_vector(1)
) * np.sqrt(2)

In [11]:
Z[:10]

array([-3.42302502e-18,  3.85784639e-14, -1.57883552e-15, -1.64234609e-03,
       -4.72544736e-16,  1.06669079e-03, -4.46803957e-14,  2.44743348e-14,
       -4.21356645e-17,  9.77629284e-18])

In [12]:
Y[:10]

array([ 3.42302502e-18, -3.85784639e-14,  1.57883552e-15,  1.64234609e-03,
        4.72544736e-16, -1.06669079e-03,  4.46803957e-14, -2.44743348e-14,
        4.21356645e-17, -9.77629284e-18])